In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
a = {'a': [1,2,3],
     'b': [4,5,6]}

In [3]:
df = pd.DataFrame(a)
df

,a,b
0,1,4
1,2,5
2,3,6


In [4]:
b = {'a': [7,8,9],
     'b': [10,11,12]}

In [5]:
df2 = pd.DataFrame(b)
df2

,a,b
0,7,10
1,8,11
2,9,12


In [9]:
df3 = pd.concat([df, df2], axis=0)
df3.reset_index(inplace=True, drop=True)
df3

,a,b
0,1,4
1,2,5
2,3,6
3,7,10
4,8,11
5,9,12
